In [1]:
import os
import sys
import json

w_dir = %pwd
work_dir = os.path.dirname(w_dir)
work_dir

'/work'

In [2]:
sys.path.append(work_dir+'/fgc_support_retri/')
from fgc_support_retri.train import *
from fgc_support_retri.utils import *
from fgc_support_retri import config

I0415 03:14:38.831227 140438341736256 file_utils.py:39] PyTorch version 1.1.0 available.


In [3]:
# train_documents = json_load(config.FGC_TRAIN) + json_load(config.FGC_DEV) + json_load(config.FGC_TEST)

In [4]:
train_documents = json_load(config.FGC_TRAIN)

In [5]:
len(train_documents)

882

In [ ]:
train_hierarchy_model(10, 12, "20200414_hierarchy+sf+amatch_plus", "sf+amatch_plus", 2e-5, train_documents=train_documents)

I0415 03:14:41.745248 140438341736256 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at /root/.cache/torch/transformers/8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
I0415 03:14:42.639292 140438341736256 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-config.json from cache at /root/.cache/torch/transformers/8a3b1cfe5da58286e12a0f5d7d182b8d6eca88c08e26c332ee3817548cf7e60a.3767c74c8ed285531d04153fe84a0791672aff52f7249b27df341dbce09b8305
I0415 03:14:42.642595 140438341736256 configuration_utils.py:169] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_r

dev_set indexing...


  0%|          | 0/882 [00:00<?, ?it/s]

train_set indexing...
D076Q09
{'text': '为川普集团前任董事长兼总裁及川普娱乐公司的创办人，', 'start': 81, 'end': 106, 'IE': {'NER': [{'id': 'D0-S0-M0', 'string': '川普集团', 'type': 'ORG', 'char_b': 1, 'char_e': 5}, {'id': 'D0-S0-M1', 'string': '董事长', 'type': 'TITLE', 'char_b': 7, 'char_e': 10}, {'id': 'D0-S0-M2', 'string': '总裁', 'type': 'TITLE', 'char_b': 11, 'char_e': 13}, {'id': 'D0-S0-M3', 'string': '川普娱乐公司', 'type': 'ORG', 'char_b': 14, 'char_e': 20}], 'COREF': {}, 'RELATION': [], 'TOKEN': [{'word': '为', 'char_b': 0, 'char_e': 1, 'pos': 'P'}, {'word': '川普', 'char_b': 1, 'char_e': 3, 'pos': 'NR'}, {'word': '集团', 'char_b': 3, 'char_e': 5, 'pos': 'NN'}, {'word': '前任', 'char_b': 5, 'char_e': 7, 'pos': 'JJ'}, {'word': '董事长', 'char_b': 7, 'char_e': 10, 'pos': 'NN'}, {'word': '兼', 'char_b': 10, 'char_e': 11, 'pos': 'CC'}, {'word': '总裁', 'char_b': 11, 'char_e': 13, 'pos': 'NN'}, {'word': '及', 'char_b': 13, 'char_e': 14, 'pos': 'CC'}, {'word': '川普', 'char_b': 14, 'char_e': 16, 'pos': 'NR'}, {'word': '娱乐', 'char_b': 16

  0%|          | 0/2619 [00:00<?, ?it/s]

...

  1%|          | 2/239 [00:00<00:12, 18.31it/s]

lr = 0.000008
epoch 5 train_loss: 0.029


100%|██████████| 239/239 [00:18<00:00, 12.60it/s]


{'sp_em': 0.159, 'sp_prec': 0.584, 'sp_recall': 0.648, 'sp_f1': 0.558}
epoch 5 eval_recall: 0.648 eval_f1: 0.558


 89%|████████▉ | 2332/2619 [07:49<00:55,  5.16it/s]